# Save model state_dict only

In [1]:
import torch
import torch.nn as nn

In [2]:
path = 'runs/train/exp_copy/weights/'

In [3]:
model_file_path = path + 'best.pt'
ckpt = torch.load(model_file_path, map_location='cpu')
torch.save(ckpt['model'], path + 'model.pt')

In [4]:
# Save state_dict only
torch.save(ckpt['model'].state_dict(), path + 'model_state_dict.pt')

In [5]:
from models.yolo import Model

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = Model('models/yolov5m.yaml', ch=3, nc=80, anchors=None).to(device)  # create


                 from  n    params  module                                  arguments                     
  0                -1  1      5280  models.common.Conv                      [3, 48, 6, 2, 2]              
  1                -1  1     41664  models.common.Conv                      [48, 96, 3, 2]                
  2                -1  2     65280  models.common.C3                        [96, 96, 2]                   
  3                -1  1    166272  models.common.Conv                      [96, 192, 3, 2]               
  4                -1  4    444672  models.common.C3                        [192, 192, 4]                 
  5                -1  1    664320  models.common.Conv                      [192, 384, 3, 2]              
  6                -1  6   2512896  models.common.C3                        [384, 384, 6]                 
  7                -1  1   2655744  models.common.Conv                      [384, 768, 3, 2]              
  8                -1  2   4134912  

In [6]:
model.load_state_dict(torch.load(path + "model_state_dict.pt", map_location='cpu'))

<All keys matched successfully>

In [9]:
batch_size = 1

model.eval()
model.to('cpu')
# Input to the model
x = torch.randn(batch_size, 3, 640, 640, requires_grad=True).to('cpu')
torch_out = model(x)

# Export the model
torch.onnx.export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  path + "modified_add.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=13,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output0'], # the model's output names
                  dynamic_axes=None    # variable length axes
                                )

/home/hongbing/Projects/yolov5/models/yolo.py:219: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if augment:
/home/hongbing/Projects/yolov5/models/yolo.py:126: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if profile:
/home/hongbing/Projects/yolov5/models/yolo.py:130: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if visualize:
/home/ho

In [15]:
import onnx

# Checks
model_onnx = onnx.load(path + "modified_add.onnx")  # load onnx model
onnx.checker.check_model(model_onnx)  # check onnx model
    
onnx.save(model_onnx, path + "modified_add.onnx")

# Simplify
try:    
    import onnxsim
    
    model_onnx, check = onnxsim.simplify(model_onnx)    
    onnx.save(model_onnx, path + "modified_add.onnx")
except Exception as e:
    print(e)

In [20]:
tensor1 = torch.randn(1, 2, 3, 3)
tensor2 = torch.randn(1, 3, 3, 3)

class Concat(nn.Module):
    # Concatenate a list of tensors along dimension
    def __init__(self, dimension=1):
        super().__init__()
        self.d = dimension

    def forward(self, *x):
        return torch.cat(x, self.d)

m = Concat()        

m.f = 2
m(tensor1, tensor2)

tensor([[[[-1.36965e+00, -1.02389e+00,  3.68400e-01],
          [-1.18623e-01, -1.05423e+00, -1.89012e-01],
          [-3.45186e-01, -1.05892e+00,  1.57285e+00]],

         [[ 7.25538e-01,  8.03549e-02,  3.21692e-01],
          [ 2.48363e-01,  1.58795e+00, -1.19322e-01],
          [ 2.41320e-01,  1.38358e+00, -1.34646e+00]],

         [[ 3.61976e-02,  4.89959e-01,  8.89310e-03],
          [ 1.17609e+00, -8.19324e-01,  4.88001e-01],
          [-1.05081e+00,  9.68253e-01, -1.16043e-03]],

         [[-2.76780e-01,  7.48895e-01,  1.18751e+00],
          [ 2.81068e-02,  2.10410e+00, -8.68632e-01],
          [ 5.69884e-01,  1.94860e+00,  2.85901e-02]],

         [[ 7.82166e-01,  4.49962e-01,  1.31943e+00],
          [-8.50161e-01,  1.90999e+00, -2.07294e-01],
          [-1.91950e-01,  4.30786e-01, -3.34761e-02]]]])

In [21]:
m2 = nn.Sequential(m)
print(m2)

Sequential(
  (0): Concat()
)


In [27]:
for name, i in m2.named_children():
    i.t = 2
    print(i)
    print(i.f)
    print(i.t)

Concat()
2
2
